## Using different DR technique (ICA) on saved neural network 

In [3]:
!pip install scipy

# Set up

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune
import sklearn.manifold as nonlin
import copy
import pickle as pk


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4
#cifar10
trainset_cifar = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset_cifar, validset_cifar = torch.utils.data.random_split(trainset_cifar, [45000,5000])



trainloader_cifar = torch.utils.data.DataLoader(trainset_cifar, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
validloader_cifar = torch.utils.data.DataLoader(validset_cifar, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset_cifar = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader_cifar = torch.utils.data.DataLoader(testset_cifar, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes_cifar = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# MNIST DATASET
mnist_train = torchvision.datasets.MNIST(root='./data', train=True,
                                         download=True, transform=transform)
mnist_train, mnist_valid = torch.utils.data.random_split(mnist_train, [50000,10000])
mnist_test = torchvision.datasets.MNIST(root='./data', train=False,
                                        download=True, transform=transform)

trainloader_mnist = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                                shuffle=True, num_workers=2)
validloader_mnist =  torch.utils.data.DataLoader(mnist_valid, batch_size=batch_size,
                                                shuffle=True, num_workers=2)                                          
testloader_mnist = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
classes_mnist =  ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')


Files already downloaded and verified
Files already downloaded and verified


In [3]:
mnist_dataset_sample = iter(testloader_mnist)
sample_im, sample_lbl = next(mnist_dataset_sample)

cifar_dataset_sample = iter(testloader_cifar)
sample_im_c, sample_lbl_c = next(cifar_dataset_sample)


# Necessary functions to perform dimensional reduction operations

In [4]:
def are_weights(model,weights):
    w1,w2,w3 = weights
    model.fc1.weight.data = w1
    model.fc2.weight.data = w2
    model.fc3.weight.data = w3
    return model


def construct_model(model, layers):
    l1, l2, l3 = layers
    model.fc1 = l1
    model.fc2 = l2
    model.fc3 = l3
    return model
    
def change_dimensionality(weight, dr_method):
    x1 = weight.detach().numpy().T
    x1 = dr_method.fit_transform(x1)
    x1 = torch.tensor(x1.T, dtype=torch.float32)
    print("x_new vector")
    print(x1.shape)
    return x1

    

In [5]:
def drprojection(model, method, whiten):
    weight_list = []
    weight1 = model.fc1.weight
    print('fc1 layer weight')

    fc1_reduced = change_dimensionality(
        weight1, method(n_components=int(weight1.shape[0]/2),  whiten=whiten, random_state=0, tol= 10))
    weight_list.append(fc1_reduced)
    x2 = torch.mm(model.fc1.weight, fc1_reduced.T)
    print(x2.shape)
    weight2 = torch.mm(model.fc2.weight, x2)
    print(weight2.shape)

    fc2_reduced = change_dimensionality(
        weight2, method(n_components=int(weight2.shape[0]/2),whiten=whiten, random_state=0, tol= 10))
    print('fc2 layer weight')
    print(fc2_reduced.shape)
    weight_list.append(fc2_reduced)

    x3_reduced = torch.mm(weight2, fc2_reduced.T)

    fc3_reduced = torch.mm(model.fc3.weight, x3_reduced)
    print('fc3 layer weight')
    print(fc3_reduced.shape)
    weight_list.append(fc3_reduced)

    return weight_list


# Average accuracy for test data

In [6]:
def average_accuracy(net, dataset):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in dataset:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(
        f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


# Accuracy for every class

In [7]:
def every_class_accuracy(model, testloader, classes):
    cor_pred = {classname: 0 for classname in classes}
    t_pred = {classname: 0 for classname in classes}
    with torch.no_grad():
        for data in testloader:
            im, labels = data
            output = model(im)
            _, predictions = torch.max(output, dim=1)
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    cor_pred[classes[label]] += 1
                t_pred[classes[label]] += 1
    
    for classname, correct_count in cor_pred.items():
        accuracy = 100 * float(correct_count) / t_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

        

# Training function 

In [28]:
print(torch.cuda.is_available())

True


In [25]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim


def training(model, trainset,valset, n, filename):
    device = torch.device('cuda:0')
    running_loss = 0.0
    total = 0.0
    correct = 0.0
    loss_nodr = []
    acc_nodr = []
    
    val_loss_nodr = []
    model = model.to(device)
    val_acc_nodr = []
  

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    print(optimizer)

    for epoch in range(n):

        print('epoch:', epoch)
        for i, data in enumerate(trainset):

            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad(True)
            with torch.set_grad_enabled(True):
                with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
                 
                 outputs = model(inputs).to(device)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                running_loss += loss.item()
        model.eval()
        val_loss = 0.0
        total_val=0.0
        correct_val = 0.0

        for i, data in enumerate(valset):

            inputs, labels2 = data
            inputs = inputs.to(device)
            labels2 = labels2.to(device)
            val_output  = model(inputs).to(device)
            loss = criterion(val_output, labels2)
            val_loss+=loss.item()
            total_val+=labels2.size(0)
            _, pred = torch.max(val_output.data, 1)
            correct_val += (pred == labels2).sum().item()

            
            
        train_loss = running_loss/len(trainset)
        acc_temp = 100 * correct / total

        valid_loss_temp = val_loss/len(valset)
        valid_acc_temp = (100 * correct_val)/total_val
        loss_nodr.append(train_loss)
        acc_nodr.append(acc_temp)

        val_loss_nodr.append(valid_loss_temp)
        val_acc_nodr.append(valid_acc_temp)
        print(
            f'[{epoch + 1}, {i + 1:5d}] train loss: {train_loss:.3f} train acc: {acc_temp:.3f}', 
            f'valid acc: {valid_acc_temp:.3f} valid loss  {valid_loss_temp:.3f} ')
        
        running_loss = 0.0
        correct = 0.0
        total = 0.0

    print('Finished Training')
    pickle_accloss(acc_nodr, loss_nodr,val_acc_nodr,val_loss_nodr,  filename)

    
def pickle_accloss(acc, loss,valid_acc, valid_loss, filename):
    accandloss = {'accuracy' : acc, 'loss' : loss, 'valid acc' : valid_acc, 'valid loss': valid_loss}
    with open('./data_vis/ICA/{}.pickle'.format(filename), 'wb') as ica_acc_and_loss:
        pk.dump(accandloss, ica_acc_and_loss)
    print('Saved dictionary of loss and accuracy!')



# Dimensional reduction with the dataset MNIST

## Loading saved network/Initializing network

In [9]:
class NN(nn.Module):
    def __init__(self, is_normalize):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(256, 120) # 256x 60
        self.fc2 = nn.Linear(120, 84) # 60 x 84
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0)
        self.norm = is_normalize

        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        if(self.norm==True):
         x = nn.functional.normalize(self.fc3(x))
        else:
         x = self.fc3(x)
         
        # x = self.soft(x)
        return x
    

In [19]:
path_original_mnist = './ica_reduced_network/mnist/original_network_mnist.pt'
# original_network_mnist = torch.load(path_original_mnist)
# original_network_mnist.eval()

In [15]:
original_network_mnist = NN(is_normalize=False)

In [27]:
training(original_network_mnist,trainloader_mnist, validloader_mnist, 5, 'mnist_original_network_accloss')

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


KeyboardInterrupt: 

In [16]:
average_accuracy(original_network_mnist, testloader_mnist)


Accuracy of the network on the 10000 test images: 99 %


In [17]:
every_class_accuracy(original_network_mnist, testloader_mnist, classes_mnist)


Accuracy for class: 0     is 99.2 %
Accuracy for class: 1     is 99.5 %
Accuracy for class: 2     is 99.0 %
Accuracy for class: 3     is 98.6 %
Accuracy for class: 4     is 98.8 %
Accuracy for class: 5     is 99.2 %
Accuracy for class: 6     is 98.7 %
Accuracy for class: 7     is 99.5 %
Accuracy for class: 8     is 99.5 %
Accuracy for class: 9     is 98.1 %


In [20]:
torch.save(original_network_mnist, path_original_mnist)

## Here we perform dimensional reduction of the original network

In [21]:
ica_mnist_network = copy.deepcopy(original_network_mnist)


In [22]:
mnist_weight_list = drprojection(
    original_network_mnist, dec.FastICA, 'unit-variance')
ica_mnist_layers = (nn.Linear(256, 60, False), nn.Linear(60, 42, False), nn.Linear(42, 10, False))
ica_mnist_network = construct_model(ica_mnist_network, ica_mnist_layers)
ica_mnist_network = are_weights(ica_mnist_network, tuple(mnist_weight_list))
ica_mnist_network.eval()

fc1 layer weight
x_new vector
torch.Size([60, 256])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [23]:
average_accuracy(ica_mnist_network, testloader_mnist)

Accuracy of the network on the 10000 test images: 77 %


In [24]:
every_class_accuracy(ica_mnist_network, testloader_mnist, classes_mnist)

Accuracy for class: 0     is 82.1 %
Accuracy for class: 1     is 55.4 %
Accuracy for class: 2     is 62.6 %
Accuracy for class: 3     is 84.6 %
Accuracy for class: 4     is 95.6 %
Accuracy for class: 5     is 81.4 %
Accuracy for class: 6     is 96.7 %
Accuracy for class: 7     is 39.4 %
Accuracy for class: 8     is 96.5 %
Accuracy for class: 9     is 87.2 %


In [25]:
path_ica_mnist_network = './ica_reduced_network/mnist/ica_mnist_network.pt'
torch.save(ica_mnist_network, path_ica_mnist_network)


## RETRAINING DR NETWORK

In [26]:
ica_mnist_network_retrained = copy.deepcopy(ica_mnist_network)
ica_mnist_network_retrained.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [27]:
training(ica_mnist_network_retrained,trainloader_mnist, validloader_mnist, 5, 'ica_accuracy_loss')

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2500] train loss: 0.086 train acc: 97.416 valid acc: 98.170 valid loss  0.063 
epoch: 1
[2,  2500] train loss: 0.043 train acc: 98.608 valid acc: 98.490 valid loss  0.050 
epoch: 2
[3,  2500] train loss: 0.030 train acc: 99.086 valid acc: 98.560 valid loss  0.054 
epoch: 3
[4,  2500] train loss: 0.025 train acc: 99.216 valid acc: 98.670 valid loss  0.063 
epoch: 4
[5,  2500] train loss: 0.021 train acc: 99.354 valid acc: 98.740 valid loss  0.051 
Finished Training
Saved dictionary of loss and accuracy!


In [28]:
average_accuracy(ica_mnist_network_retrained, testloader_mnist)

Accuracy of the network on the 10000 test images: 98 %


In [29]:
every_class_accuracy(ica_mnist_network_retrained, testloader_mnist, classes_mnist)

Accuracy for class: 0     is 99.4 %
Accuracy for class: 1     is 99.7 %
Accuracy for class: 2     is 99.4 %
Accuracy for class: 3     is 99.4 %
Accuracy for class: 4     is 99.3 %
Accuracy for class: 5     is 98.9 %
Accuracy for class: 6     is 96.3 %
Accuracy for class: 7     is 99.3 %
Accuracy for class: 8     is 99.1 %
Accuracy for class: 9     is 97.7 %


In [30]:
path_ica_mnist_retrained = './ica_reduced_network/mnist/ica_mnist_retrained_network.pt'
torch.save(ica_mnist_network_retrained, path_ica_mnist_retrained)


## REINITIALIZING SMALL NETWORK AND RETRAIING IT

In [31]:
small_layer_mnist = (nn.Linear(400,60), nn.Linear(60,42), nn.Linear(42,10))
small_network_mnist = NN(is_norm=False)
small_network_mnist = construct_model(small_network_mnist,small_layer_mnist)

In [32]:
training(small_network_mnist, trainloader_mnist,
         validloader_mnist, 5, 'small_network_mnist_accloss')


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  2500] train loss: 0.261 train acc: 91.630 valid acc: 97.930 valid loss  0.067 
epoch: 1
[2,  2500] train loss: 0.060 train acc: 98.186 valid acc: 98.470 valid loss  0.051 
epoch: 2
[3,  2500] train loss: 0.043 train acc: 98.614 valid acc: 98.760 valid loss  0.044 
epoch: 3
[4,  2500] train loss: 0.033 train acc: 98.992 valid acc: 98.660 valid loss  0.047 
epoch: 4
[5,  2500] train loss: 0.027 train acc: 99.164 valid acc: 98.960 valid loss  0.037 
Finished Training
Saved dictionary of loss and accuracy!


In [33]:
average_accuracy(small_network_mnist, testloader_mnist)


Accuracy of the network on the 10000 test images: 98 %


In [34]:
path_small_network = './ica_reduced_network/mnist/small_mnist_network.pt'
torch.save(small_network_mnist, path_small_network)


## Pruning the original network

In [39]:
pruned_original_network_mnist = copy.deepcopy(original_network_mnist)

### first we create a function

In [40]:
def prune_function(model, amount_to_prune):
    module_lits = []
    for module in model.modules():
        if isinstance(module, nn.Linear):
            module_lits.append((module, 'weight'))
    prune.global_unstructured(
        module_lits,
        pruning_method=prune.RandomUnstructured,
        amount = amount_to_prune
    )

## perform pruning by calling a function

In [41]:
prune_function(pruned_original_network_mnist,0.5)

In [42]:
average_accuracy(pruned_original_network_mnist, testloader_mnist)

Accuracy of the network on the 10000 test images: 89 %


In [44]:
every_class_accuracy(pruned_original_network_mnist, testloader_mnist, classes_mnist)

Accuracy for class: 0     is 94.9 %
Accuracy for class: 1     is 95.9 %
Accuracy for class: 2     is 88.5 %
Accuracy for class: 3     is 95.0 %
Accuracy for class: 4     is 98.4 %
Accuracy for class: 5     is 93.6 %
Accuracy for class: 6     is 96.2 %
Accuracy for class: 7     is 98.2 %
Accuracy for class: 8     is 95.7 %
Accuracy for class: 9     is 39.8 %


In [45]:
path_pruned_original_network_mnist = './ica_reduced_network/mnist/pruned/pruned_original_network_mnist.pt'
torch.save(pruned_original_network_mnist,path_pruned_original_network_mnist)

## Dimensional reduction and Pruning of original network mnist

In [46]:
ica_pruned_network_mnist = copy.deepcopy(
    pruned_original_network_mnist)


In [47]:
average_accuracy(ica_pruned_network_mnist, testloader_mnist)


Accuracy of the network on the 10000 test images: 89 %


In [48]:
mnist_weight_list_pruned = drprojection(
    ica_pruned_network_mnist, dec.FastICA, 'unit-variance')
ica_mnist_layers_pruned = (nn.Linear(256, 60, False), nn.Linear(
    60, 42, False), nn.Linear(42, 10, False))
ica_pruned_network_mnist = construct_model(
    ica_pruned_network_mnist, ica_mnist_layers_pruned)
ica_pruned_network_mnist = are_weights(ica_pruned_network_mnist, tuple(mnist_weight_list_pruned))
ica_pruned_network_mnist.eval()


fc1 layer weight
x_new vector
torch.Size([60, 256])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [49]:
average_accuracy(ica_pruned_network_mnist, testloader_mnist)


Accuracy of the network on the 10000 test images: 66 %


In [50]:
every_class_accuracy(ica_pruned_network_mnist, testloader_mnist, classes_mnist)

Accuracy for class: 0     is 91.9 %
Accuracy for class: 1     is 30.7 %
Accuracy for class: 2     is 43.1 %
Accuracy for class: 3     is 86.9 %
Accuracy for class: 4     is 90.6 %
Accuracy for class: 5     is 82.2 %
Accuracy for class: 6     is 97.4 %
Accuracy for class: 7     is 51.9 %
Accuracy for class: 8     is 71.4 %
Accuracy for class: 9     is 24.4 %


In [51]:
path_ica_pruned_network_mnist = './ica_reduced_network/mnist/pruned/ica_pruned_network_mnist.pt'
torch.save(ica_pruned_network_mnist, path_ica_pruned_network_mnist)


# Dimensional reduction using the data CIFAR10

In [22]:
class NN2(nn.Module):
    def __init__(self, is_norm=False):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.soft = nn.Softmax(dim=0)
        self.is_norm = is_norm
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        if(self.is_norm == True):
            x = nn.functional.normalize(self.fc3(x))
        else:
            x = self.fc3(x)
        return x


In [54]:
path_cifar_original_net = './ica_reduced_network/cifar/cifar_original_net.pt'
# cifar_original_net = torch.load(path_cifar_original_net)
# cifar_original_net.eval()

In [23]:
cifar_original_net = NN2(is_norm=False)

In [24]:
training(cifar_original_net, trainloader_cifar, validloader_cifar, 10, 'cifar_original_net_accloss')

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  1250] train loss: 1.694 train acc: 37.722 valid acc: 46.020 valid loss  1.467 
epoch: 1
[2,  1250] train loss: 1.333 train acc: 52.118 valid acc: 53.620 valid loss  1.309 
epoch: 2
[3,  1250] train loss: 1.180 train acc: 57.829 valid acc: 59.780 valid loss  1.178 
epoch: 3
[4,  1250] train loss: 1.078 train acc: 61.720 valid acc: 61.200 valid loss  1.128 
epoch: 4
[5,  1250] train loss: 1.006 train acc: 64.387 valid acc: 61.800 valid loss  1.097 
epoch: 5
[6,  1250] train loss: 0.941 train acc: 66.791 valid acc: 61.620 valid loss  1.144 
epoch: 6
[7,  1250] train loss: 0.890 train acc: 68.324 valid acc: 62.900 valid loss  1.095 
epoch: 7
[8,  1250] train loss: 0.844 train acc: 69.860 valid acc: 61.920 valid loss  1.134 
epoch: 8
[9,  1250] train loss: 0.808 train acc: 71.529 valid acc: 63.900 val

In [57]:
average_accuracy(cifar_original_net, testloader_cifar)

Accuracy of the network on the 10000 test images: 64 %


In [58]:
every_class_accuracy(cifar_original_net, testloader_cifar, classes_cifar)

Accuracy for class: plane is 67.1 %
Accuracy for class: car   is 73.3 %
Accuracy for class: bird  is 52.4 %
Accuracy for class: cat   is 30.0 %
Accuracy for class: deer  is 58.7 %
Accuracy for class: dog   is 66.5 %
Accuracy for class: frog  is 77.7 %
Accuracy for class: horse is 66.7 %
Accuracy for class: ship  is 76.9 %
Accuracy for class: truck is 72.1 %


In [59]:
torch.save(cifar_original_net, path_cifar_original_net)


## Dimensional reduction of cifar network

In [65]:
ica_cifar_network = copy.deepcopy(cifar_original_net)

cifar_weight_list = drprojection(
    ica_cifar_network, dec.FastICA, 'unit-variance')

cifar_mnist_layers = (nn.Linear(256, 60, False), nn.Linear(
    60, 42, False), nn.Linear(42, 10, False))

ica_cifar_network = construct_model(
    ica_cifar_network, cifar_mnist_layers)
    
ica_cifar_network = are_weights(ica_cifar_network, tuple(cifar_weight_list))
ica_cifar_network.eval()


fc1 layer weight
x_new vector
torch.Size([60, 400])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [66]:
average_accuracy(ica_cifar_network, testloader_cifar)

Accuracy of the network on the 10000 test images: 32 %


In [67]:
every_class_accuracy(ica_cifar_network, testloader_cifar, classes_cifar)

Accuracy for class: plane is 24.7 %
Accuracy for class: car   is 73.3 %
Accuracy for class: bird  is 25.5 %
Accuracy for class: cat   is 0.3 %
Accuracy for class: deer  is 26.2 %
Accuracy for class: dog   is 17.6 %
Accuracy for class: frog  is 94.0 %
Accuracy for class: horse is 6.6 %
Accuracy for class: ship  is 38.9 %
Accuracy for class: truck is 13.7 %


In [68]:
path_ica_cifar_network ='./ica_reduced_network/cifar/ica_cifar_network.pt'
torch.save(ica_cifar_network, path_ica_cifar_network)

## Retraining of the ica network

In [69]:
ica_cifar_network_retrained = copy.deepcopy(ica_cifar_network)

In [70]:
training(ica_cifar_network_retrained, trainloader_cifar, validloader_cifar, 10, 'ica_cifar_retrained_accloss')

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  1250] train loss: 1.438 train acc: 50.176 valid acc: 54.080 valid loss  1.340 
epoch: 1
[2,  1250] train loss: 1.200 train acc: 58.736 valid acc: 59.160 valid loss  1.187 
epoch: 2
[3,  1250] train loss: 1.116 train acc: 62.280 valid acc: 61.200 valid loss  1.147 
epoch: 3
[4,  1250] train loss: 1.061 train acc: 63.733 valid acc: 62.460 valid loss  1.119 
epoch: 4
[5,  1250] train loss: 1.013 train acc: 65.184 valid acc: 62.280 valid loss  1.141 
epoch: 5
[6,  1250] train loss: 0.983 train acc: 66.389 valid acc: 63.480 valid loss  1.121 
epoch: 6
[7,  1250] train loss: 0.951 train acc: 67.822 valid acc: 62.700 valid loss  1.142 
epoch: 7
[8,  1250] train loss: 0.921 train acc: 68.569 valid acc: 63.660 valid loss  1.139 
epoch: 8
[9,  1250] train loss: 0.897 train acc: 69.233 valid acc: 63.920 valid loss  1.090 
epoch: 9
[10,  1250] train loss: 0.879 train acc: 69.931 valid acc: 64.220 valid loss  1.107 
Finished Training
Saved dictionary of loss and accuracy!


In [72]:
average_accuracy(ica_cifar_network_retrained, testloader_cifar)

Accuracy of the network on the 10000 test images: 63 %


In [73]:
every_class_accuracy(ica_cifar_network_retrained, testloader_cifar, classes_cifar)

Accuracy for class: plane is 51.0 %
Accuracy for class: car   is 82.7 %
Accuracy for class: bird  is 55.5 %
Accuracy for class: cat   is 45.6 %
Accuracy for class: deer  is 54.8 %
Accuracy for class: dog   is 50.4 %
Accuracy for class: frog  is 76.5 %
Accuracy for class: horse is 66.4 %
Accuracy for class: ship  is 75.4 %
Accuracy for class: truck is 75.4 %


In [74]:
path_ica_cifar_network_retrained = './ica_reduced_network/cifar/ica_cifar_network_retrained.pt'
torch.save(ica_cifar_network_retrained, path_ica_cifar_network_retrained)

## Small cifar network training


In [75]:
small_layer = (nn.Linear(400,60), nn.Linear(60,42), nn.Linear(42,10))
small_network_cifar = NN2(is_norm=False)
small_network_cifar = construct_model(small_network_cifar,small_layer)

In [76]:
training(small_network_cifar, trainloader_cifar, validloader_cifar, 10,'small_network_cifar_accloss')

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  1250] train loss: 1.734 train acc: 35.433 valid acc: 45.820 valid loss  1.467 
epoch: 1
[2,  1250] train loss: 1.345 train acc: 51.427 valid acc: 55.900 valid loss  1.231 
epoch: 2
[3,  1250] train loss: 1.185 train acc: 57.856 valid acc: 60.320 valid loss  1.130 
epoch: 3
[4,  1250] train loss: 1.081 train acc: 61.993 valid acc: 62.720 valid loss  1.079 
epoch: 4
[5,  1250] train loss: 1.013 train acc: 64.522 valid acc: 64.340 valid loss  1.050 
epoch: 5
[6,  1250] train loss: 0.953 train acc: 66.669 valid acc: 66.000 valid loss  0.991 
epoch: 6
[7,  1250] train loss: 0.918 train acc: 67.958 valid acc: 63.480 valid loss  1.089 
epoch: 7
[8,  1250] train loss: 0.885 train acc: 69.238 valid acc: 65.600 valid loss  1.018 
epoch: 8
[9,  1250] train loss: 0.856 train acc: 70.267 valid acc: 66.640 valid loss  0.983 
epoch: 9
[

In [77]:
average_accuracy(small_network_cifar, testloader_cifar)

Accuracy of the network on the 10000 test images: 64 %


In [78]:
every_class_accuracy(small_network_cifar, testloader_cifar, classes_cifar)

Accuracy for class: plane is 64.3 %
Accuracy for class: car   is 72.2 %
Accuracy for class: bird  is 58.3 %
Accuracy for class: cat   is 49.0 %
Accuracy for class: deer  is 64.7 %
Accuracy for class: dog   is 52.1 %
Accuracy for class: frog  is 68.6 %
Accuracy for class: horse is 67.5 %
Accuracy for class: ship  is 83.5 %
Accuracy for class: truck is 68.0 %


In [79]:
path_small_network_cifar = './ica_reduced_network/cifar/small_network_cifar.pt'
torch.save(small_network_cifar, path_small_network_cifar)

## Pruning with cifar dataset

In [80]:
pruned_cifar_net = copy.deepcopy(cifar_original_net)

In [81]:
prune_function(pruned_cifar_net, 0.25)

In [82]:
average_accuracy(pruned_cifar_net, testloader_cifar)

Accuracy of the network on the 10000 test images: 54 %


In [83]:
path_pruned_cifar_net = './ica_reduced_network/cifar/pruned/pruned_cifar_net.pt'
torch.save(pruned_cifar_net, path_pruned_cifar_net)

## dimensional reduction and prunning with cifar10

In [84]:
ica_pruned_net_cifar = copy.deepcopy(pruned_cifar_net)

cifar_weight_list_pruned = drprojection(
    ica_pruned_net_cifar, dec.FastICA, 'unit-variance')

pruned_cifar_mnist_layers = (nn.Linear(256, 60, False), nn.Linear(
    60, 42, False), nn.Linear(42, 10, False))

ica_pruned_net_cifar = construct_model(
    ica_pruned_net_cifar, pruned_cifar_mnist_layers)

ica_pruned_net_cifar = are_weights(ica_pruned_net_cifar, tuple(cifar_weight_list_pruned))
ica_pruned_net_cifar.eval()



fc1 layer weight
x_new vector
torch.Size([60, 400])
torch.Size([120, 60])
torch.Size([84, 60])
x_new vector
torch.Size([42, 60])
fc2 layer weight
torch.Size([42, 60])
fc3 layer weight
torch.Size([10, 42])


NN2(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=60, bias=False)
  (fc2): Linear(in_features=60, out_features=42, bias=False)
  (fc3): Linear(in_features=42, out_features=10, bias=False)
  (soft): Softmax(dim=0)
)

In [85]:
average_accuracy(ica_pruned_net_cifar, testloader_cifar)

Accuracy of the network on the 10000 test images: 36 %


In [86]:
every_class_accuracy(ica_pruned_net_cifar, testloader_cifar, classes_cifar)

Accuracy for class: plane is 54.9 %
Accuracy for class: car   is 52.5 %
Accuracy for class: bird  is 21.2 %
Accuracy for class: cat   is 11.3 %
Accuracy for class: deer  is 44.9 %
Accuracy for class: dog   is 32.8 %
Accuracy for class: frog  is 32.7 %
Accuracy for class: horse is 10.0 %
Accuracy for class: ship  is 33.0 %
Accuracy for class: truck is 72.9 %


In [87]:
path_ica_pruned_net_cifar = './ica_reduced_network/cifar/pruned/ica_pruned_net_cifar.pt'
torch.save(ica_pruned_net_cifar,path_ica_pruned_net_cifar)